In [220]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import librosa
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [221]:
def extract_mfccs(file_path):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    mfccs = mfccs.T
    delta_mfccs = librosa.feature.delta(mfccs)
    delta_mfccs = delta_mfccs
    
    #(216) rows = frames ,  and (26) columns = features
    features = np.hstack([mfccs, delta_mfccs])
    feature_names = [f"MFCC_{i+1}" for i in range(features.shape[1])]
    return pd.DataFrame(features, columns=feature_names)


In [222]:
def extract_pitch(file_path):
    y, sr = librosa.load(file_path)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitch = np.mean(pitches[magnitudes > np.max(magnitudes) * 0.85])
    return pd.DataFrame({"Pitch": [pitch]})

In [223]:
def extract_chroma(file_path):
    y, sr = librosa.load(file_path)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma = chroma.T
    feature_names = [f"Chroma_{i+1}" for i in range(chroma.shape[1])]
    return pd.DataFrame(chroma, columns=feature_names)

In [224]:
def extract_zero_crossings(file_path):
    y, sr = librosa.load(file_path)
    zero_crossings = librosa.feature.zero_crossing_rate(y)
    zero_crossings = zero_crossings.T
    feature_names = [f"ZeroCrossings_{i+1}" for i in range(zero_crossings.shape[1])]
    return pd.DataFrame(zero_crossings, columns=feature_names)

In [225]:
def extract_spectral_contrast(file_path):
    y, sr = librosa.load(file_path)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast = spectral_contrast.T 
    feature_names = [f"SpectralContrast_{i+1}" for i in range(spectral_contrast.shape[1])]
    return pd.DataFrame(spectral_contrast, columns=feature_names)

In [226]:
def process_user_folder(folder_path):
    mfccs_list = []
    pitch_list = []
    chroma_list = []
    zero_crossings_list = []
    spectral_contrast_list = []
    flag = 1

    # Iterate over all .wav files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".wav"):
            file_path = os.path.join(folder_path, file_name)
            if flag:
                mfccs_list = extract_mfccs(file_path)
                pitch = extract_pitch 
                pitch_list.append(pitch)
                chroma_list = extract_chroma(file_path)
                zero_crossings_list = extract_zero_crossings(file_path)
                spectral_contrast_list = extract_spectral_contrast(file_path)

                flag = 0
            else:
                # Extract features for each file
                mfccs = extract_mfccs(file_path)
                pitch = extract_pitch(file_path)
                chroma = extract_chroma(file_path)
                zero_crossings = extract_zero_crossings(file_path)
                spectral_contrast = extract_spectral_contrast(file_path)
                # Append features to the lists
                mfccs_list = pd.concat([mfccs,mfccs_list]) 
                chroma_list = pd.concat([chroma, chroma_list])
                zero_crossings_list = pd.concat([zero_crossings, zero_crossings_list])
                spectral_contrast_list = pd.concat([spectral_contrast, spectral_contrast_list])
                pitch_list.append(pitch)


    # Create feature matrix with separate columns for each feature type
    #Data set without the pitch
    compined_features = pd.concat([mfccs_list, chroma_list, zero_crossings_list, spectral_contrast_list], axis=1)

    return compined_features

In [242]:
# Specify the path of the main folder
main_folder_path = "voices"

# Get a list of all subfolders inside the main folder
subfolders = [f.path for f in os.scandir(main_folder_path) if f.is_dir()]
# create a list of two train datas one for voice recognition (ahmed_ali, bedro, hassan, muhannad) and the other for speech recognition (grant_me_access,unlock_the_gate,open_middle_door)
data_voice_recognition_features = {
    "ahmed_ali": [],
    "bedro": [],
    "hassan": [],
    "muhannad": []

}
data_speech_recognition_features = {
    "grant_me_access": [],
    "open_middle_door": [],
    "unlock_the_gate": []
}

# Loop over each subfolder
for subfolder_path , i in zip(subfolders , range(len(subfolders))) :
    # Extract the name of the subfolder
    subfolder_name = os.path.basename(subfolder_path)
    print(subfolder_name)
    
    # Create a variable with the features of the subfolder
    features = process_user_folder(subfolder_path)

    # Assign the variable name based on the subfolder name
    globals()[subfolder_name] = features
    if subfolder_name == "ahmed_ali" or subfolder_name == "bedro" or subfolder_name == "hassan" or subfolder_name == "muhannad" or subfolder_name == "abdullah_ahmed" or subfolder_name == "ashf" or subfolder_name == "atef" or subfolder_name == "hazem_rafaat":
        data_voice_recognition_features[subfolder_name] = features
    else:
        data_speech_recognition_features[subfolder_name] = features

abdulla_ahmed
ahmed_ali
ashf
atef
bedro
grant_me_access
hassan
hazem_rafaat
muhannad
open_middle_door
unlock_the_gate
Voices


In [244]:
# Combine all the extracted features into a single DataFrame
all_features_speech = pd.concat([data_speech_recognition_features["grant_me_access"], data_speech_recognition_features["open_middle_door"], data_speech_recognition_features["unlock_the_gate"]])
# Create labels for the features based on the subfolder names
labels = ["grant_me_access"] * data_speech_recognition_features["grant_me_access"].shape[0] + ["open_middle_door"] * data_speech_recognition_features["open_middle_door"].shape[0] + ["unlock_the_gate"] * data_speech_recognition_features["unlock_the_gate"].shape[0]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features_speech, labels, test_size=0.2, random_state=42)



In [ ]:
# Initialize the SVM model
svm_model_speech = SVC(kernel='linear')

# Train the SVM model
svm_model_speech.fit(X_train, y_train)



In [235]:
# Make predictions on the test set
predictions = svm_model_speech.predict(X_test)



In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.bestparams
best_svm_model = grid_search.bestestimator


In [ ]:
from sklearn import metrics

y_pred = best_svm_model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average='weighted')
recall = metrics.recall_score(y_test, y_pred, average='weighted')
f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

In [236]:
# Evaluate the performance of the model
accuracy = accuracy_score(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.6455566905005107
Classification Report:
               precision    recall  f1-score   support

   ahmed_ali       0.64      0.67      0.65       482
       bedro       0.69      0.67      0.68       526
      hassan       0.60      0.64      0.62       571
    muhannad       0.67      0.59      0.63       379

    accuracy                           0.65      1958
   macro avg       0.65      0.64      0.65      1958
weighted avg       0.65      0.65      0.65      1958


In [237]:
# do the same for voice recognition
# Combine all the extracted features into a single DataFrame
all_features = pd.concat([data_voice_recognition_features["ahmed_ali"], data_voice_recognition_features["bedro"], data_voice_recognition_features["hassan"], data_voice_recognition_features["muhannad"], data_voice_recognition_features["abdullah_ahmed"], data_voice_recognition_features["ashf"], data_voice_recognition_features["atef"], data_voice_recognition_features["hazem_rafaat"]])
# Create labels for the features based on the subfolder names
labels = ["ahmed_ali"] * data_voice_recognition_features["ahmed_ali"].shape[0] + ["bedro"] * data_voice_recognition_features["bedro"].shape[0] + ["hassan"] * data_voice_recognition_features["hassan"].shape[0] + ["muhannad"] * data_voice_recognition_features["muhannad"].shape[0] + ["abdullah_ahmed"] * data_voice_recognition_features["abdullah_ahmed"].shape[0] + ["ashf"] * data_voice_recognition_features["ashf"].shape[0] + ["atef"] * data_voice_recognition_features["atef"].shape[0] + ["hazem_rafaat"] * data_voice_recognition_features["hazem_rafaat"].shape[0]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, labels, test_size=0.2, random_state=42)

In [238]:
# Initialize the SVM model
svm_model_voice = SVC(kernel='linear')


In [239]:
# Train the SVM model
svm_model_voice.fit(X_train, y_train)



SVC(kernel='linear')

In [240]:
# Make predictions on the test set
predictions = svm_model_voice.predict(X_test)



In [241]:
# Evaluate the performance of the model
accuracy = accuracy_score(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.6455566905005107
Classification Report:
               precision    recall  f1-score   support

   ahmed_ali       0.64      0.67      0.65       482
       bedro       0.69      0.67      0.68       526
      hassan       0.60      0.64      0.62       571
    muhannad       0.67      0.59      0.63       379

    accuracy                           0.65      1958
   macro avg       0.65      0.64      0.65      1958
weighted avg       0.65      0.65      0.65      1958
